In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup
from urllib import parse
import urllib.request as req
import time
import pandas as pd


import math

In [2]:
# Chrome Browser 와 Chrome Driver Version 확인하기
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
driver.get('https://paikdabang.com/store/')


<!-- 시도 클릭-> i 클릭 -> 구군 클릭 -> j 클릭 -> next버튼 체크, 있으면 계속  -->
전체-전체로 검색, 총 1256개의 매장 확인, 10으로 나눠서 125번 넥스트 버튼, 6개만 따로 

In [3]:
#max_count
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')
maxCount = soup.select_one('#max_count')
maxCount = maxCount.string

maxCount = int(maxCount)
maxPage = math.ceil(maxCount/10)
print(maxPage)


126


In [13]:
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')
shopName = []
shopAddress=[]
shopArea=[]
errorList =[]
count = 2
shops = soup.select('.tr_list')
# shops

for i in range(0,maxPage):
# for i in range(1,10):
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    shops = soup.select('.tr_list')
    for shop in shops:
        # print(shop.select_one('.td_area').text)
        # print(shop.select_one('.td_shop').text)
        # print(shop.select_one('.td_addr').text)
        shopName.append(shop.select_one('.td_shop').text)
        shopArea.append(shop.select_one('.td_area').text)
        shopAddress.append(shop.select_one('.td_addr').text.replace(",",""))
    try: driver.find_element(By.XPATH, '//*[@id="pagination"]/li['+ str(count) +']').click()
    except: errorList.append('Error number : '+ i)
    else:
        time.sleep(1)
        if count < 6:
            count += 2
        else:
            count = 6
    


In [5]:
errorList

[]

In [12]:
shopName = shopArea = shopAddress = []


---
# Save the CSV

In [14]:
df = pd.DataFrame(data=shopArea, columns=['Area']) #< columns는 [] 로 작성해야댐 
df['ShopName'] = pd.Series(shopName)
df['Address'] = pd.Series(shopAddress)

In [10]:
df

,Area,ShopName,Address
0,경기도,화성남양시장점,경기 화성시 남양읍 남양성지로 139-1
1,대구광역시,대구수성3가점,대구 수성구 수성동3가 72-1
2,전라북도,전주대자인병원점,전주시 덕진구 우아동3가749-63
3,인천광역시,인천아인여성병원점,인천광역시 미추홀구 주안동 포레나미추홀 201동 1층 1032호
4,서울특별시,월곡두산위브점,서울시 성북구 오패산로 46
...,...,...,...
1255,울산광역시,울산남목초교점,"울산시 동구 동해안로 28, 1층"
1256,경기도,용인동백쥬네브점,"경기도 용인시 기흥구 동백중앙로 283, 골드프라자 D121호"
1257,서울특별시,신림역2호점,서울시 관악구 신림동 1641-12
1258,서울특별시,청량리역점,"서울시 동대문구 왕산로 209, 1층"


In [15]:
df.to_csv('../Data/PaiksCoffee.csv', encoding='utf-8', index=False)